# CNN & Transfer Learning



**Lernziele**

---
* Sie verstehen, was mit Transfer Learning gemeint ist.
* Sie können vortrainierte Modelle laden und anpassen.
* Sie können Bilder aus Ordnern in Python einlesen.
---

Letzte Woche haben Sie ein CNN entwickelt, das Zahlen erkennen kann. Heute geht es darum, die Vorteile von bereits trainierten Netzwerken zu nutzen. Zunächst werden Sie ein Modelltrainieren, das zwischen Hunde- und Katzenrassen unterscheiden kann. In der Übungsaufgabe werden Sie ResNet verwenden, um Lungenentzündung in Röntgenbildern zu erkennen.

Beim Transfer Lerning wird ein bereits trainiertes Modell für ein anderes Machine Learning Problem verwendet. 

<img align="center" src="https://pennylane.ai/qml/_images/transfer_learning_general.png" width="400">
<h6 align="center">Pennylane.ai</h6>


Das pre-trained Modell ist meist ein Modell, das mit vielen allgemeinen Daten trainiert wurde. Dadurch konnte das Modell genügend generelle Informationen lernen, welche auch für sehr spezifisches Problem relevant sein können.

ResNet wurde zum Beispiel mit den Daten von ImageNet trainiert. Diese enthalten keine Röntgenbilder. Aber durch die Kombination von bereits trainierten Layers des ResNet-Modells und neuen, nicht trainierten Layers können wir das "Wissen" von ResNet nutzen. 

Das Trainieren dieser Modelle kann diese Woche dauern!



---

**Das heutige Training wird viel länger dauern als bisher. Wenn Sie dieses Notebook auf Google Collab laufen lassen, können Sie einfach einen GPU verwenden, um das Training zu beschleunigen. Damit Sie auf einem GPU trainieren können, müssen Sie einfach den folgenden Anweisungen folgen:**

Wählen Sie oben auf der Website die folgenden Menüs aus.

```
Lauftzeit > Laufzeittyp ändern > Hardwarebeschleuniger 
```

Falls Ihr Google Collab auf Englisch ist:

```
Runtime > Change runtime type > Hardware Accelerator
```

Wählen Sie hier *GPU*.

Wenn Sie dieses Notebook lokal (auf Ihrem eigenen Rechner) laufen lassen, müssen Sie PyTorch mit Cuda-Unterstützung und der richtigen Version von Cuda auf Ihrem System installieren, um einen GPU zu verwenden. Dies funktioniert nur, wenn Sie eine GPU in Ihr Laptop oder PC über einen GPU verfügt. Aber dieses Notebook funktioniert auch, wenn Sie keinen Zugang zu einem GPU haben, es wird nur etwas länger dauern.

---

In [ ]:
from __future__ import print_function, division
!pip install rdkit==2022.3.4
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch import sigmoid
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import sys
%matplotlib inline
if 'google.colab' in sys.modules: # checks whether the notebook runs on collab
    !pip install rdkit==2022.3.4
    !wget https://raw.githubusercontent.com/kochgroup/intro_pharma_ai/main/utils/utils.py
    %run utils.py
else:
    %run ../utils/utils.py # lädt vorgeschriebene Funktionen

plt.ion()


In [ ]:
# Der folgende Code lädt die Daten für das Notebook herunter, kann etwas länger dauern
if 'google.colab' in sys.modules:
    !wget https://uni-muenster.sciebo.de/s/TaOR0Lk50rjPHUU/download
    !unzip -q download -d  ../
    !rm download
else:
    import wget
    import zipfile
    wget.download("https://uni-muenster.sciebo.de/s/TaOR0Lk50rjPHUU/download")
    with zipfile.ZipFile("data.zip","r") as zip_ref:
        zip_ref.extractall("../")

MNIST ist ein relativ kleiner Datensatz und kann daher auf einmal in den Speicher geladen werden. Wie in der Vorlesung besprochen, sind Bilder jedoch normalerweise größer als die Zahlen in MNIST. Um mit großen Bilddatensätzen umgehen zu können, hat PyTorch seine eigene Library `torchvision`. Hier sind wichtige Funktionen enthalten, die wir im normalen `torch` nicht haben. 

Wichtig zu beachten ist auch, wie die Daten nun gespeichert werden. Wenn Sie in den Ordner `data/images_animals/` navigieren, sehen Sie zwei Ordner. Der erste Ordner `train` enthält die Trainingsbilder. Der zweite Ordner `val` enthält die Testdaten. Innerhalb dieser Ordner gibt es wiederum Unterordner, die nach den Labels der Bilder benannt sind. Das bedeutet, dass z.B. der Ordner `beagel` nur Bilder von Beageln enthält.

Wenn genau eine solche Ordnerstruktur existiert, können wir die Daten sehr einfach mit `torchvision` einlesen.
Aber bevor wir die Daten einlesen können, müssen wir Transformationen der Bilder definieren. 

Erstens sind die Bilder zu groß. Die meisten vortrainierten Modelle erwarten eine Bildgröße von 224 x 224 Pixeln, da dies die Größe der Bilder im ImageNet-Datensatz ist. Außerdem müssen die Bilder noch in einen `tensor` umgewandelt werden. In einem letzten Schritt skalieren wir die Daten. Diesmal verwenden wir nicht den `minmax`-Skalierer, sondern normalisieren die Bilder. **Dazu werden der Mittelwert und die Standardabweichung der ImageNet-Bilder verwendet. Denn mit diesen Bildern (deren Mittelwertzen) wurde das Netzwerk trainiert.

Die Funktion `transforms.Compose()` funktioniert ähnlich wie `nn.Sequential`. Auf alle Bilder werden nacheinander alle Transformationen angewandt.

In [ ]:
data_transforms = transforms.Compose([
        transforms.Resize((224,224)), #reduziert die Größe des Bildes
        transforms.ToTensor(), #konvertiert das Bild zu einem Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]) #Normalisiert die Bilder 

Nun, da die Transformationen definiert sind, können Sie einen PyTorch-Datensatz erstellen. Diesmal verwenden wir aber die spezielle Klasse `datasets.ImageFolder`. Diese spezielle Klasse `dataset` ist genau auf unsere Ordnerstruktur abgestimmt. Wir müssen nur den `path` zu den Bildern angeben und welche Transformationen wir anwenden wollen.

In [ ]:
train_data = datasets.ImageFolder('../data/images_animals/train',data_transforms)
test_data = datasets.ImageFolder('../data/images_animals/val',data_transforms)
train_data

Sie können sehen, dass wir insgesamt 5913 Bilder in unserem Trainingsordner haben. Außerdem sind die Transformationen, die angewendet werden sollen, aufgelistet.

Als letzten Schritt erstellen wir den `DataLoader`. Diesmal auch für den Testdatensatz, da wir nicht alle Bilder auf einmal "ins Netzwerk laden" können und daher auch die Evaluierung in Batches erfolgen müssen.

In [ ]:
torch.manual_seed(1235)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16,
                                             shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=16,
                                             shuffle=True)

example_batch = datasets.ImageFolder('../data/example_batch',data_transforms)
example_batch = torch.utils.data.DataLoader(example_batch, batch_size=6,shuffle=False)

Sie wissen bis jetzt noch nicht, welche und wie viele verschiedene Klasse wir haben. Wir können uns aus dem Datensatz diese Information holen:

In [ ]:
class_names = train_data.classes
print(class_names)
print("\nAnzahl Klassen: ",len(class_names))

# Wir speichern einen Batch, um diesen besser zu analysieren
inputs_example, targets_example = next(iter(example_batch))

Insgesamt haben wir 37 verschiedene Arten von Hunden und Katzen. Wir können uns die Bilder auch mit einer vorgeschriebenen Funktion anschauen.

In [ ]:
out = torchvision.utils.make_grid(inputs_example[:6])
imshow(out, title=["birman", "birman", "persian", "persian", "pug", "sphynx"])

### RESNET

Sie haben jetzt die Daten im richtigen Format. Bevor wir jedoch mit dem Training beginnen können, müssen wir auch unser Modell in das richtige Format bringen. 
Wie bereits erwähnt, bietet PyTorch mehrere Modelle, die bereits trainiert wurden. Diese können einfach geladen werden. *Wenn Sie ein Modell zum ersten Mal laden, müssen die Gewichte noch aus dem Internet geladen werden, was einige Zeit dauern kann.*

Wir verwenden auch nur ResNet18, da alle größeren Netze zu langsam wären, um sie auf dem Uniserver zu trainieren.

In [ ]:
resnet18 = models.resnet18(pretrained=True)
resnet18

`resnet18` gibt Ihnen eine Übersicht, welche PyTorch Layers in welcher Reihenfolgen benutzt werden. Achten Sie vor allem auf die letzte Layer mit dem Namen `fc`. Diese können wir auch direkt mit `resnet.fc` auswählen.

In [ ]:
resnet18.fc

Diese Layer ist eine `nn.Linear`-Layer, die Sie aus dem PyTorch-Einführungs Notebook kennen sollten. Sie hat 512 Features als Inputgröße und 1000 als Output. Diese 1000 Outputneuronen entsprechen den 1000 verschiedenen Klassen im ImageNet-Datensatz.

Um das "ResNet"-Modell weiter vorzubereiten, wollen wir zunächst alle Layers des ResNet einfrieren. Das bedeutet, dass diese Layers keine Weightupdates erhalten und somit nicht weiter trainiert werden können. Wir können das machen, da das Modell bereits trainiert wurde.
Der folgende Code iteriert durch alle Schichten und setzt `requires_grad` auf `False`. Das lässt PyTorch wissen, dass für diese Layers keine Gradienten berechnet werden müssen. 

In [ ]:
for param in resnet18.parameters():
    param.requires_grad = False

Als letzes müssen wir nur noch die `fc` Layer austauschen. Da wir nicht 1000 Klassen, sondern 37 haben. Also brauchen wir eine neue `nn.Linear` Layer, die als Input die Größe 512 hat und als Output die Größe 37.

In [ ]:
torch.manual_seed(1234)
resnet18.fc = nn.Linear(512, 37) #ersetzten der Linear Layer

print(resnet18.fc)
list(resnet18.fc.parameters())

Sie können erkennen, dass die neue `fc` Layer `requires_grad=True` gesetzt hat. Das bedeutet diese Weights werden während des Trainings geupdatet. Also ist die `fc` Layer die einzige zu trainiernde Layer im Netzwerk.

## Training 

Jetzt können wir mit dem Trainingsloop beginnen. Doch zunächst definieren wir die Loss Funktion und den Optimizer.

In [ ]:
loss_funktion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.0001)

Die Trainingsloop wird heute etwas komplexer aussehen. Das liegt daran, dass wir jetzt auch das Testset als Batches durch das Netzwerk führen. Um die Metriken trotzdem korrekt zu berechnen, haben wir `running_loss` und `running_corrects` verwendet, um sie am Ende des Loops zu mitteln.
Der Trainingsprozess wird aufgrund der vielen Berechnungen ziemlich lange dauern, selbst wenn nur eine Layer trainiert wird.


Der folgende Code prüft, ob ein GPU verfügbar ist. Wenn dies der Fall ist, findet das gesamte Training auf dem GPU statt, andernfalls wird auf dem CPU trainiert.

```python
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
```

Um auf der GPU zu trainieren, müssen wir das Modell und die Daten auf den GPU verschieben. Dies geschieht mit: `.to(device)` 

```python
resnet18.to(device)
```
Schließlich müssen die Batches während des Trainings auch auf den GPU verschoben werden:

```python
inputs = inputs.to(device)
targets = targets.to(device)
```


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet18.to(device)

torch.manual_seed(3333)
for epoch in range(3):
    #### Training ####
    resnet18.train()
    running_loss = 0
    running_corrects = 0
    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        output=resnet18(inputs)
        _ , preds = torch.max(output, 1)
        loss = loss_funktion(output,targets)
        running_loss +=loss.item()
        loss.backward()
        optimizer.step()
        running_corrects +=torch.sum(preds == targets.data).cpu() 
    epoch_loss = running_loss/len(train_loader)    
    epoch_acc = running_corrects.double() / len(train_data)  
    print('Trainings Loss: {:.4f} Trainings Acc: {:.4f}'.format(
        epoch_loss, epoch_acc))
    
    
    #### Evaluierung #####
    resnet18.eval()
    running_loss = 0
    running_corrects = 0
    for inputs, targets in tqdm(test_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        output=resnet18(inputs)
        _ , preds =torch.max(output, 1)
        loss = loss_funktion(output,targets)
        running_loss +=loss.item()
        running_corrects +=torch.sum(preds == targets.data).cpu() 
    epoch_acc = running_corrects.double() / len(test_data) 
    epoch_loss = running_loss/len(test_loader)    
    print('Test Loss: {:.4f} Test Acc: {:.4f}'.format(
        epoch_loss, epoch_acc))

Nach drei Epochen erreichen wir bereits eine Testaccuracy von 0,8. 80 % der Bilder werden richtig klassifiziert.

Um wirklich sicher zu gehen, dass das Pre-Training des Modells einen Unterschied gemacht hat, trainieren wir dasselbe Modell noch einmal. Diesmal jedoch ohne die vortrainierten Weights zu laden:

`pretrained=False`

In [ ]:
resnet18 = models.resnet18(pretrained=False) #  <- ResNet wird ohne die vortrainierten Weights geladen
torch.manual_seed(1234)
resnet18.fc = nn.Linear(512, 37)
loss_funktion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.0001)
torch.manual_seed(3333)
for epoch in range(3):
    
    #### Training ####
    resnet18.train()
    running_loss = 0
    running_corrects = 0
    for inputs, targets in tqdm(train_loader):
        optimizer.zero_grad()
        output=resnet18(inputs)
        _ , preds = torch.max(output, 1)
        loss = loss_funktion(output,targets)
        running_loss +=loss.item()
        loss.backward()
        optimizer.step()
        running_corrects +=torch.sum(preds == targets.data)   
    epoch_loss = running_loss/len(train_loader)    
    epoch_acc = running_corrects.double() / len(train_data)  
    print('Trainings Loss: {:.4f} Trainings Acc: {:.4f}'.format(
        epoch_loss, epoch_acc))
    
    #### Evaluierung #####
    resnet18.eval()
    running_loss = 0
    running_corrects = 0
    for inputs, targets in tqdm(test_loader):
        output=resnet18(inputs)
        _ , preds =torch.max(output, 1)
        loss = loss_funktion(output,targets)
        running_loss +=loss.item()
        running_corrects +=torch.sum(preds == targets.data)
    epoch_acc = running_corrects.double() / len(test_data) 
    epoch_loss = running_loss/len(test_loader)    
    print('Test Loss: {:.4f} Test Acc: {:.4f}'.format(
        epoch_loss, epoch_acc))

Tatsächlich sind wir nach 3 Epochen nicht annähernd so genau, als wenn wir das "vortrainierte" Modell verwendet hätten. Das liegt daran, dass die pretrained Convolutions eine Art Feature-Generierung durchführen.

Wir können dies deutlicher sehen, wenn wir uns die Aktivierungen der Convolution ansehen. 
Dazu verwenden wir die Beispielbilder vom Anfang dieses Notebooks.

In [ ]:
out = torchvision.utils.make_grid(inputs_example[:6])
imshow(out, title=["birman", "birman", "persian", "persian", "pug", "sphynx"])

Zunächst laden wir wieder das vortrainierte Resnet Model. Auch hier entfernen wir die `fc` Layer, aber ersetzen diese nicht durch eine neue linear Layer. Dadurch haben wir direkt Zugriff auf den Output der Convolution Layer. Diese Model nennen wir `resnet_convolutions`.

In [ ]:
resnet18 = models.resnet18(pretrained=True)
resnet_convolutions = nn.Sequential(*list(resnet18.children())[:-1])
resnet_convolutions.eval()

Als Letztes führen wir die 6 Bilder von gerade eben durch dieses spezielle Netzwerk und speichern den Output (`feature_encoding`). Dieser Output dient als der Input für die linear Layer, die wir trainieren würden.

<img align="center" src="Img/transferlearning/tf_2.png" width="500">

In [ ]:
feature_encodings=resnet_convolutions(inputs_example)[:6,:,0,0]
feature_encodings

Diese "Encodings" sollen eine Art reduzierte Darstellung des Originalbildes sein. Eine Art Fingerprint. Wenn es stimmt, dass die vortrainierten Convolutions bestimmte Merkmale finden, die für die Klassifizierung relevant sind, dann sollten ähnliche Bilder auch ähnliche Merkmale aufweisen.

Zum Beispiel zeigen das dritte und vierte Bild jeweils eine "persische" Katze. Also sollten auch die Encodings der Bilder ähnlich sein. Wir können mit Hilfe der `cosine_similarity` beurteilen, wie ähnlich zwei Vektoren sind. Die Werte liegen immer zwischen -1 (sehr unähnlich) und 1 (sehr ähnlich). 
Wir können die Ähnlichkeit zwischen dem dritten Bild (`persian`) und allen anderen Bildern berechnen.

In [ ]:
cosine_similarity(feature_encodings[2:3].detach(),feature_encodings.detach()).round(3)

Die Ähnlichkeit des dritten Bildes zum dritten Bild ist natürlich `1`, weil es das selbe Bild ist. Aber zu den anderen Bildern ist die Ähnlichkeit geringer. Am ähnlichsten ist das vierte Bild mit `0.891`, dieses Bild ist ebenfalls ein Bild einer `persian` Katze. Das bedeutet, dass dieses vortrainierte Modell bereits in der Lage war, bestimmte Ähnlichkeiten in den Bildern zu erkennen.   

>Aber die Bilder könnten auch schon vor den Convolutions ähnlich sein?

Das ist richtig, aber wir können auc das überprüfen. In der folgenden Zelle berechnen wir die Ähnlichkeit der Originalbilder vor den Convolutions.

In [ ]:
cosine_similarity(inputs_example.flatten(1)[2:3],inputs_example.flatten(1)[0:6]).round(3)

Hier fällt auf, dass das zweite Bild einer `persian`  Katze das unähnlichste ist, obwohl auf beiden Bildern die gleiche Katzenart zu sehen ist. Wir können also schlussfolgern, dass das Netzwerk in der Tat ähnliche Fetaures in Bildern finden kann.

**ImageNet, der Datensatz, auf dem ResNet ursprünglich trainiert wurde, enthält auch verschiedene Katzen- und Hunderassen, darunter auch die Rasse "Perser". Die Auswirkungen des Pretrainings werden wahrscheinlich weniger ausgeprägt sein, wenn es um die Klassifizierung von Rassen geht, die für ResNet völlig "neu" sind.**

Zum Schluss probieren wir aus, wie gut unser Netzwerk funktioniert, wenn wir das vortrainierte Modell laden und unsere eigene lineare Schicht erstellen. Diesmal frieren wir jedoch die vortrainierten Convolution Layers nicht ein, sondern trainieren diese auch noch weiter.  

In [ ]:
resnet18 = models.resnet18(pretrained=True) #PRETRAIN = TRUE
torch.manual_seed(1234)
resnet18.fc = nn.Linear(512, 37) 
loss_funktion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.0001)
torch.manual_seed(3333)
for epoch in range(3):
    
    #### Training ####
    resnet18.train()
    running_loss = 0
    running_corrects = 0
    for inputs, targets in tqdm(train_loader):
        optimizer.zero_grad()
        output=resnet18(inputs)
        _ , preds = torch.max(output, 1)
        loss = loss_funktion(output,targets)
        running_loss +=loss.item()
        loss.backward()
        optimizer.step()
        running_corrects +=torch.sum(preds == targets.data)   
    epoch_loss = running_loss/len(train_loader)    
    epoch_acc = running_corrects.double() / len(train_data)  
    print('Trainings Loss: {:.4f} Trainings Acc: {:.4f}'.format(
        epoch_loss, epoch_acc))
    
    #### Evaluierung #####
    resnet18.eval()
    running_loss = 0
    running_corrects = 0
    for inputs, targets in tqdm(test_loader):
        output=resnet18(inputs)
        _ , preds =torch.max(output, 1)
        loss = loss_funktion(output,targets)
        running_loss +=loss.item()
        running_corrects +=torch.sum(preds == targets.data)
    epoch_acc = running_corrects.double() / len(test_data) 
    epoch_loss = running_loss/len(test_loader)    
    print('Test Loss: {:.4f} Test Acc: {:.4f}'.format(
        epoch_loss, epoch_acc))

Dieses Netz führt zu den besten Ergebnissen. Das liegt daran, dass nun auch die Weights der Convolutions weiter trainiert werden können. So wird auch die Feeaturegenerierung besser auf unseren Datensatz abgestimmt. 

In der Praxis werden oft unterschiedliche Lernraten für die neue lineare Layers und die bereits trainierten Convolutions verwendet. Dadurch kann die neue lineare Layer schneller trainiert werden als die Convolutions. 

# Übungsaufagbe

Bitte starten Sie den Kernel erneut, bevor Sie die Übung durchführen.

Wie in der Vorlesung mehrfach besprochen, benutzen wir heute für die Übung ein vortrainiertes Modell zur Erkennung einer Lungenentzündung an Hand von Röntgenbildern.

Dazu müssen Sie die Daten korrekt einlesen, das Modell vorbereiten und die `for-loop` ausfüllen.

In [ ]:
from __future__ import print_function, division
from torch.nn.functional import sigmoid
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch import sigmoid
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from __future__ import print_function, division
from torch.nn.functional import sigmoid
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch import sigmoid
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import sys
if 'google.colab' in sys.modules:
    !pip install rdkit==2022.3.4
    !wget https://raw.githubusercontent.com/kochgroup/intro_pharma_ai/main/utils/utils.py
    %run utils.py
else:
    %run ../utils/utils.py

plt.ion()
plt.ion()

Navigieren Sie zunächst zu dem Ordner, in dem die Tierbilder bereits gespeichert sind. Dort finden Sie auch einen Ordner `chest_xray`. Dieser Ordner enthält ebenfalls Unterordner mit den jeweiligen Trainings- und Testdatensätzen.
Legen Sie zunächst fest, welche Transformationen auf die Bilder angewendet werden sollen.

In [ ]:
data_transforms = transforms.Compose([
        transforms.Resize((224,224)), #reduziert die Größe des Bildes
        transforms.ToTensor(), #konvertiert das Bild zu einem Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]) #Normalisiert die Bilder 

Als Nächstes laden Sie die entsprechenden `Datasets` und `DataLoader`:

In [ ]:
train_data = datasets.ImageFolder('../data/chest_xray/train',data_transforms)
test_data = datasets.ImageFolder('../data/chest_xray/val',data_transforms)
train_data

In [ ]:
torch.manual_seed(1235)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=True)

Finden Sie heraus, wie viele verschiedene Klassen wir haben, und denken Sie daran, dass dies die Definition unserer Lossfunktion und unseres Netzwerks beeinflusst.

In [ ]:
class_names = train_data.classes
print(class_names)
print("\nAnzahl Klassen: ",len(class_names))

In [ ]:
inputs_example, targets_example = next(iter(train_loader))

out = torchvision.utils.make_grid(inputs_example[:2])
imshow(out, title=[class_names[x] for x in targets_example[:2]])

Laden Sie zunächst das **vortrainierte** `resnet18`

In [ ]:
resnet18 = models.resnet18(____________)


Verhindern Sie, dass die `resnet` Layers noch weiter trainiert werden:

In [ ]:
for param in resnet18.parameters():
    __________ = ____________

Ersetzen Sie die richtige Layer mit einer neuen Layer.

In [ ]:
torch.manual_seed(1234)
______ = ___________

Definieren Sie Lossfunktion und Optimizer. Welche Lossfunktion sollten wir für diese Anzahl an Klassen nehmen?

In [ ]:
loss_funktion = __________
optimizer = optim.Adam(_______________, lr=0.001)

Füllen Sie als letztes den Trainingsloop aus. Wir benutzen `type_as(output)` um die richtigen `dtype` zuerhalten.

In [ ]:
torch.manual_seed(3333)
for epoch in range(3):
    
    #### Training ####
    resnet18.train()
    
    #Brauchen wir für die Loss und AUC Berechnung
    running_loss = 0
    pred_ll = []
    targets_ll = []
    
    
    for inputs, targets in tqdm(_______):
        optimizer.zero_grad()
        
        #Forward Propagation
        output=resnet18(__________).squeeze()
        loss = loss_funktion(_______ , _____.type_as(output))
        
        # Speichern des Loss und der Predictions
        pred_ll.append(sigmoid(output).squeeze().detach().clone().numpy())
        targets_ll.append(targets.detach().numpy())
        running_loss +=loss.item()
        
        # Backpropagation
        loss.backward()
        optimizer.step()
         
    epoch_loss = running_loss/len(train_loader)    
    epoch_auc =  roc_auc_score(targets_ll,pred_ll)
    print('Trainings Loss: {:.4f} Trainings AUC: {:.4f}'.format(
        epoch_loss, epoch_auc))
    
    
    #### Evaluierung #####
    resnet18.eval()
    
    #Brauchen wir für die Loss und AUC Berechnung
    running_loss = 0
    pred_ll = []
    targets_ll = []
    
    for inputs, targets in tqdm(__________):
        
        #Forward Propagation
        output=resnet18(_______).squeeze()
        loss = loss_funktion(______,______.type_as(output))
        
        pred_ll.append(sigmoid(output).squeeze().detach().clone().numpy())
        targets_ll.append(targets.detach().numpy())
        running_loss +=loss.item()

    epoch_auc =  roc_auc_score(targets_ll,pred_ll)
    epoch_loss = running_loss/len(test_loader)    
    print('Test Loss: {:.4f} Test Auc: {:.4f}'.format(
        epoch_loss, epoch_auc))